In [ ]:
# Reading data

In [ ]:
import polars as pl

data = pl.read_csv("data.csv", parse_dates=False)
data = data.with_columns(pl.col("date").str.slice(-7).alias("date_monthly"))
data = data.with_column(pl.col("date").str.strptime(pl.Date, fmt="%d/%m/%Y"))

In [ ]:
data = data[:1000000]

In [ ]:
# Text data pre-processing

In [ ]:
from cleantext import clean

def preprocess(text):
    output = clean(str(text), punct=True,
                              extra_spaces=True,
                              stopwords=True,
                              lowercase=True,
                              numbers = True)
    return output

In [ ]:
data_clean = data.with_columns([
    pl.col("headline").apply(preprocess)
])

In [ ]:
# Text data representation

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# calculate the compound score
def sentiment_vader(sentence):

    # create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)

    # create overall (compound) indicator
    compound = sentiment_dict['compound']

    return compound

In [ ]:
# apply the function with Polars

sentiment = data_clean.with_columns([
    pl.col("headline").apply(sentiment_vader)
])

In [ ]:
# Time-series representation

In [ ]:
# aggregate over months

timeseries = (sentiment.lazy()
    .groupby("date_monthly")
    .agg(
        [
            pl.avg("headline")
        ]
    ).sort("date_monthly")
).collect()


In [ ]:
# Time-series representation

In [ ]:
# aggregate over months

timeseries = (sentiment.lazy()
    .groupby("date_monthly")
    .agg(
        [
            pl.avg("headline")
        ]
    ).sort("date_monthly")
).collect()
